# Example stage reports

Example stage reports based on comparing current and previous stage.

In [1]:
from wrc_rallydj.livetiming_api import WRCLiveTimingAPIClient, time_to_seconds
from rules_processor import p, Nth
from pandas import merge, DataFrame


# The cacheing is tricky:
# - we want to be able to force updates for live stages etc
# There is internal state in the class, which stores
# the last requested data unless we force an update
# wrc = WRCLiveTimingAPIClient(use_cache=True, backend="sqlite", expire_after=600)
wrc = WRCLiveTimingAPIClient(use_cache=True, backend="memory", expire_after=600)

# wrc.initialise(year=2024)
# wrc.seasonId
# Current year, latest race by default
wrc.initialise()

In [2]:
stage_details = wrc.getStageDetails()
stage_details.head()

,id,stageNo,STAGE TYPE,stageId,eventId,STATUS,day,name,distance
0,ee55daa1-fd07-574f-835a-f39008081b95,SHD,shakedown,SHD,535,Complete,,Shakedown,NaN
1,d560e801-c242-547b-baa1-881fcf11d8d1,SS1,SpecialStage,8330,535,Completed,Thursday,SS1 Umeå Sprint 1 (5.16km),5.16
2,7b3fc9fa-348b-5afd-a956-82aab0a68058,SS2,SpecialStage,8331,535,Completed,Friday,SS2 Bygdsiljum 1 (28.27km),28.27
3,4a48fad1-cd0e-59ac-b632-4613fb242f95,SS3,SpecialStage,8332,535,Completed,Friday,SS3 Andersvattnet 1 (20.51km),20.51
4,13100067-f837-5d2d-8a75-bba344f39f7f,SS4,SpecialStage,8333,535,Completed,Friday,SS4 Bäck 1 (10.8km),10.80


In [3]:
STAGE='SS3'

wrc.getOverall(stageId="SS2").head()

,id,pos,carNo,driverId,driverCountry,driverCountryImage,driver,coDriverId,coDriverCountry,coDriverCountryImage,...,diffPrev,groupClass,championshipId,totalTimeInS,timeToCarBehind,overallGap,overallDiff,eventId,rallyId,stageId
0,a402156a-a574-503d-8992-47d081c8eba5,1,#33,ae7329c9-79b3-5d96-886c-22cca6217764,United Kingdom,Flags/GBR.png,Elfyn EVANS,53e56691-fe7c-5271-9dc5-8960df28b221,United Kingdom,Flags/GBR.png,...,,RC1,289,1040.4,2.7,0.0,0.0,535,583,8331
1,03d073d5-2e53-51dc-a25e-93497a622b35,2,#8,6632e7ca-34bf-55b8-9cad-d060000fa794,Estonia,Flags/EST.png,Ott TÄNAK,00a8a5c3-f7ba-5086-86df-1a59b7da7e26,Estonia,Flags/EST.png,...,+2.7,RC1,289,1043.1,3.8,2.7,2.7,535,583,8331
2,bf8b45f0-afea-5c55-8067-a1f1d7f10cfe,3,#18,298f93b1-b0ef-5af4-9f0c-e468d29abfd2,Japan,Flags/JPN.png,Takamoto KATSUTA,6347a621-eade-5fff-a7b4-7c4b73651e8a,Ireland,Flags/IRL.png,...,+3.8,RC1,289,1046.9,1.1,6.5,3.8,535,583,8331
3,03b5d275-58e9-5ff8-8fd2-fd5d25c54a23,4,#16,bf956e8a-5cad-5327-add0-26e0481ea508,France,Flags/FRA.png,Adrien FOURMAUX,8231e16a-f455-56fa-9b3c-dc3befa854cf,France,Flags/FRA.png,...,+1.1,RC1,289,1048.0,4.2,7.6,1.1,535,583,8331
4,4f1dc222-a7ba-5afd-bce7-12733bb7e86d,5,#69,d8e4bbea-3af2-5486-9ad5-a445aaec573e,Finland,Flags/FIN.png,Kalle ROVANPERÄ,e4dd8a3f-00e9-59f7-9871-9337af6085d7,Finland,Flags/FIN.png,...,+4.2,RC1,289,1052.2,2.1,11.8,4.2,535,583,8331


In [4]:
wrc.getStageTimes(stageId=STAGE).head()

,id,pos,carNo,driverId,driverCountry,driverCountryImage,driver,coDriverId,coDriverCountry,coDriverCountryImage,...,championshipId,eventId,rallyId,Gap,Diff,timeInS,timeToCarBehind,speed (km/h),pace (s/km),pace diff (s/km)
0,d71b21a3-0e54-502c-8aa6-0ef5fbd8b153,1,#16,bf956e8a-5cad-5327-add0-26e0481ea508,France,Flags/FRA.png,Adrien FOURMAUX,8231e16a-f455-56fa-9b3c-dc3befa854cf,France,Flags/FRA.png,...,289,535,583,0.0,0.0,643.8,-1.8,114.687791,31.389566,0.000000
1,ffc62051-dc3e-5fae-80a5-506b2581d64f,2,#33,ae7329c9-79b3-5d96-886c-22cca6217764,United Kingdom,Flags/GBR.png,Elfyn EVANS,53e56691-fe7c-5271-9dc5-8960df28b221,United Kingdom,Flags/GBR.png,...,289,535,583,1.8,1.8,645.6,-1.2,114.368030,31.477328,0.087762
2,a5e946b4-41d7-5606-8f4e-03a9aefa3d10,3,#18,298f93b1-b0ef-5af4-9f0c-e468d29abfd2,Japan,Flags/JPN.png,Takamoto KATSUTA,6347a621-eade-5fff-a7b4-7c4b73651e8a,Ireland,Flags/IRL.png,...,289,535,583,3.0,1.2,646.8,0.0,114.155844,31.535836,0.146270
3,3e80424e-26d5-52a9-a38b-b49818ec889e,4,#8,6632e7ca-34bf-55b8-9cad-d060000fa794,Estonia,Flags/EST.png,Ott TÄNAK,00a8a5c3-f7ba-5086-86df-1a59b7da7e26,Estonia,Flags/EST.png,...,289,535,583,3.0,0.0,646.8,-1.7,114.155844,31.535836,0.146270
4,8a654f4a-fd8d-55ff-9d0c-8df626a8d62f,5,#1,c99a2a26-bd03-5153-aaa7-684d3acb5491,Belgium,Flags/BEL.png,Thierry NEUVILLE,b1b98699-0332-528a-8a80-11eb538f1ded,Belgium,Flags/BEL.png,...,289,535,583,4.7,1.7,648.5,-2.2,113.856592,31.618723,0.229157


In [5]:
# Previous
prev_idx = stage_details.loc[stage_details["stageNo"]==STAGE].index[0]
PREV_STAGE = stage_details.loc[prev_idx-1, "stageNo"] if STAGE!="SS1" and STAGE!="SHD" else ""
PREV_STAGE

'SS2'

In [47]:
def core_stage(stageNo):
    retcols_stage = ["carNo", "driver", "pos", "Gap", "Diff"]
    retcols_overall = ["carNo", "driver", "pos", "overallGap", "overallDiff", "timeToCarBehind"]

    prev_idx = stage_details.loc[stage_details["stageNo"] == stageNo].index[0]
    prevStageNo = (
        stage_details.loc[prev_idx - 1, "stageNo"]
        if STAGE != "SS1" and STAGE != "SHD"
        else ""
    )
    # print(prevStageNo, stageNo)
    _df_stage_curr = wrc.getStageTimes(stageId=stageNo)[retcols_stage].copy()

    _df_overall_curr = wrc.getOverall(stageId=stageNo, update=True)[
        retcols_overall
    ].copy()
    _df_overall_curr.rename(columns={"pos": "overallPos"}, inplace=True)
    _df_overall_prev = wrc.getOverall(stageId=prevStageNo, update=True)[
        retcols_overall
    ].copy()
    _df_overall_prev.rename(columns={"pos": "overallPos"}, inplace=True)
    # The subtraction is this way to handle signs better
    display(_df_overall_prev)
    display(_df_overall_curr)
    _df_overall_diff = (
        _df_overall_prev.set_index(["carNo", "driver"])
        - _df_overall_curr.set_index(["carNo", "driver"])
    ).reset_index()
    _df_overall_diff.rename(
        columns={
            "overallPos": "overallPosDelta",
            "overallGap": "overallGapDelta",
            "overallDiff": "overallDiffDelta",
            "timeToCarBehind": "timeToCarBehindDelta",
        },
        inplace=True,
    )
    _df_overall_curr["currPodium"] = _df_overall_curr["overallPos"] <= 3
    _df_overall = merge(_df_overall_diff, _df_overall_curr, on=["carNo", "driver"])

    _df_overall_prev["prevLeader"] = _df_overall_prev["overallPos"] == 1
    _df_overall_prev["prevOverallPos"] = _df_overall_prev["overallPos"]
    _df_overall_prev["prevPodium"] = _df_overall_prev["overallPos"] <= 3
    _df_overall = merge(
        _df_overall,
        _df_overall_prev[["carNo", "driver", "prevLeader", "prevOverallPos", "prevPodium"]],
        on=["carNo", "driver"],
    )

    _df_overall["overallPosChange"] = _df_overall["overallPosDelta"]!=0
    _df_overall["currLeader"] = _df_overall["overallPos"] == 1
    _df_overall["onto_podium"] = (~_df_overall["prevPodium"]) & _df_overall["currPodium"]
    _df_overall["lost_podium"] = (_df_overall["prevPodium"]) & ~_df_overall["currPodium"]
    _df_overall["newLeader"] = (
        _df_overall["overallPosChange"] & _df_overall["currLeader"]
    )
    _df_overall["lead_changed"] = _df_overall["newLeader"].any()

    _df_overall = merge( _df_overall, _df_stage_curr,
        on=["carNo", "driver"],
    )
    
    return (_df_overall, _df_stage_curr)

_overall_diff, _stage = core_stage("SS6")

display(_overall_diff)
display(_stage)

,carNo,driver,overallPos,overallGap,overallDiff,timeToCarBehind
0,#18,Takamoto KATSUTA,1,0.0,0.0,5.8
1,#33,Elfyn EVANS,2,5.8,5.8,3.9
2,#8,Ott TÄNAK,3,9.7,3.9,0.2
3,#16,Adrien FOURMAUX,4,9.9,0.2,3.5
4,#1,Thierry NEUVILLE,5,13.4,3.5,11.5
5,#69,Kalle ROVANPERÄ,6,24.9,11.5,25.7
6,#55,Joshua MCERLEAN,7,50.6,25.7,11.6
7,#5,Sami PAJARI,8,62.2,11.6,6.9
8,#13,Grégoire MUNSTER,9,69.1,6.9,NaN


,carNo,driver,overallPos,overallGap,overallDiff,timeToCarBehind
0,#33,Elfyn EVANS,1,0.0,0.0,1.9
1,#18,Takamoto KATSUTA,2,1.9,1.9,1.7
2,#8,Ott TÄNAK,3,3.6,1.7,5.7
3,#16,Adrien FOURMAUX,4,9.3,5.7,5.0
4,#1,Thierry NEUVILLE,5,14.3,5.0,11.8
5,#69,Kalle ROVANPERÄ,6,26.1,11.8,24.1
6,#55,Joshua MCERLEAN,7,50.2,24.1,20.7
7,#5,Sami PAJARI,8,70.9,20.7,2.5
8,#13,Grégoire MUNSTER,9,73.4,2.5,NaN


,carNo,driver,overallPosDelta,overallGapDelta,overallDiffDelta,timeToCarBehindDelta,overallPos,overallGap,overallDiff,timeToCarBehind,...,prevPodium,overallPosChange,currLeader,onto_podium,lost_podium,newLeader,lead_changed,pos,Gap,Diff
0,#1,Thierry NEUVILLE,0,-0.9,-1.5,-0.3,5,14.3,5.0,11.8,...,False,False,False,False,False,False,True,5,7.0,1.3
1,#13,Grégoire MUNSTER,0,-4.3,4.4,NaN,9,73.4,2.5,NaN,...,False,False,False,False,False,False,True,8,10.4,2.4
2,#16,Adrien FOURMAUX,0,0.6,-5.5,-1.5,4,9.3,5.7,5.0,...,False,False,False,False,False,False,True,3,5.5,5.2
3,#18,Takamoto KATSUTA,-1,-1.9,-1.9,4.1,2,1.9,1.9,1.7,...,True,True,False,False,False,False,True,7,8.0,0.7
4,#33,Elfyn EVANS,1,5.8,5.8,2.0,1,0.0,0.0,1.9,...,True,True,True,False,False,True,True,2,0.3,0.3
5,#5,Sami PAJARI,0,-8.7,-9.1,4.4,8,70.9,20.7,2.5,...,False,False,False,False,False,False,True,9,14.8,4.4
6,#55,Joshua MCERLEAN,0,0.4,1.6,-9.1,7,50.2,24.1,20.7,...,False,False,False,False,False,False,True,4,5.7,0.2
7,#69,Kalle ROVANPERÄ,0,-1.2,-0.3,1.6,6,26.1,11.8,24.1,...,False,False,False,False,False,False,True,6,7.3,0.3
8,#8,Ott TÄNAK,0,6.1,2.2,-5.5,3,3.6,1.7,5.7,...,True,False,False,False,False,False,True,1,0.0,0.0


,carNo,driver,pos,Gap,Diff
0,#8,Ott TÄNAK,1,0.0,0.0
1,#33,Elfyn EVANS,2,0.3,0.3
2,#16,Adrien FOURMAUX,3,5.5,5.2
3,#55,Joshua MCERLEAN,4,5.7,0.2
4,#1,Thierry NEUVILLE,5,7.0,1.3
5,#69,Kalle ROVANPERÄ,6,7.3,0.3
6,#18,Takamoto KATSUTA,7,8.0,0.7
7,#13,Grégoire MUNSTER,8,10.4,2.4
8,#5,Sami PAJARI,9,14.8,4.4


In [48]:
_overall_diff.columns

Index(['carNo', 'driver', 'overallPosDelta', 'overallGapDelta',
       'overallDiffDelta', 'timeToCarBehindDelta', 'overallPos', 'overallGap',
       'overallDiff', 'timeToCarBehind', 'currPodium', 'prevLeader',
       'prevOverallPos', 'prevPodium', 'overallPosChange', 'currLeader',
       'onto_podium', 'lost_podium', 'newLeader', 'lead_changed', 'pos', 'Gap',
       'Diff'],
      dtype='object')

In [49]:
leader_row = _overall_diff[_overall_diff["overallPos"] == 1].iloc[0]
leader_row

carNo                           #33
driver                  Elfyn EVANS
overallPosDelta                   1
overallGapDelta                 5.8
overallDiffDelta                5.8
timeToCarBehindDelta            2.0
overallPos                        1
overallGap                      0.0
overallDiff                     0.0
timeToCarBehind                 1.9
currPodium                     True
prevLeader                    False
prevOverallPos                    2
prevPodium                     True
overallPosChange               True
currLeader                     True
onto_podium                   False
lost_podium                   False
newLeader                      True
lead_changed                   True
pos                               2
Gap                             0.3
Diff                            0.3
Name: 4, dtype: object

In [51]:
def rule_onto_podium(row):
    remark = ""
    if row["onto_podium"]:
        remark = f"""{row["driver"]} moved into a podium position"""
    return (remark, 0.9)

def rule_into_first(row):
    remark = ""
    if row["newLeader"]:
        big_jump = f"""jumped { p.number_to_words(row["overallPosDelta"])} places""" if row["overallPosDelta"]>1 else "moved"
        remark = f"""{row["driver"]} {big_jump} into first place overall, taking a lead of {round(row["timeToCarBehind"], 1)}s"""
    return (remark, 1.0)

def rule_lost_first(row):
    remark = ""
    if row["prevLeader"] and not row["currLeader"]:
        remark = f"""Coming in at {Nth(row["pos"])} on the stage, {row["Gap"]}s behind the stage winner, {row["driver"]} lost the overall lead, falling back to {Nth(row["overallPos"])}"""
    if row["overallPos"]>=2:
        fell_back = f"""{remark}, {row["overallGap"]}s behind the new leader."""
    else:
        remark = f"{remark}."

    return (remark, 0.9)


# TO DO - need a natural time for timeInS

def rule_leader_increased_lead(row):
    remark = ""
    if row["overallGapDelta"] > 0 and row["currLeader"] and not row["newLeader"]:
        remark = f"""{row["driver"]} increased his the lead at the front of the rally, moving a furher {row["overallGapDelta"]}s ahead, to {row["overallGapDelta"]}s."""

    return (remark, 0.7)


def rule_leader_decreased_lead(row):
    remark = ""
    if row["overallGapDelta"] < 0 and row["currLeader"] and not row["newLeader"]:
        remark = f"""At the front, {row["driver"]}'s lead was reduced by   {row["overallGapDelta"]}s to {row["overallGapDelta"]}s."""

    return (remark, 0.7)

def rule_onto_podium(row):
    if row["onto_podium"]:
        remark = ""


def process_rules(df):
    # Apply each rule to create new columns
    remarks_df = DataFrame(
        {
            "podium_remarks": df.apply(rule_onto_podium, axis=1),
            "into_first_remarks": df.apply(rule_into_first, axis=1),
            "lost_first_remarks": df.apply(rule_lost_first, axis=1),
            "lead_increased_remarks": df.apply(rule_leader_increased_lead, axis=1),
            "lead_reduced_remarks": df.apply(rule_leader_decreased_lead, axis=1),
            "move_into_podium_remarks": df.apply(rule_onto_podium, axis=1),
        }
    )

    # Stack all remarks into a single series and filter out empty strings
    filtered_remarks = [(remark[0].replace("  "," "), remark[1]) for remark in remarks_df.stack() if remark[0] != ""]

    return filtered_remarks

remarks = process_rules(_overall_diff)
remarks

[('Coming in at seventh on the stage, 8.0s behind the stage winner, Takamoto KATSUTA lost the overall lead, falling back to second',
  0.9),
 ('Elfyn EVANS moved into first place overall, taking a lead of 1.9s', 1.0),
 ('.', 0.9)]